In [8]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import utils
from pathlib import Path

sys.path.append("../../../analysisTools/")
import plotTools as ptools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [3]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

## All Lxy Bin

In [4]:
prefix = './coffea/skimmed/'
suffix = '_ctau-1-BDT-based_AllLxy'

In [5]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [6]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_ctau-1-BDT-based_Background_Efficiency_ctau-1_AllLxyBin'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_ctau-1-BDT-based_Background_Efficiency_ctau-1_AllLxyBin.csv


In [10]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,0.33
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,0.78
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39",34.95
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76",10.09
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50",46.15


In [11]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_ctau-1-BDT-based_Background_EventCount_Weighted_ctau-1_AllLxyBin'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_ctau-1-BDT-based_Background_EventCount_Weighted_ctau-1_AllLxyBin.csv


In [12]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,12311,381,160,159,159,158,3,0
Top,49818,4688,3058,2348,2337,2336,2006,3
DY,205,13,11,11,11,11,9,0
Diboson,18067,1234,915,860,852,848,767,3
Triboson,2257,170,116,106,105,104,92,0
W+jets,697657,48425,37015,35472,35415,35377,29090,110
Z+jets,1281459,67427,52627,50986,50934,50903,44480,75
Total,2061774,122338,93902,89942,89813,89737,76447,191


In [13]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_ctau-1-BDT-based_Background_EventCount_Raw_ctau-1_AllLxyBin'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_ctau-1-BDT-based_Background_EventCount_Raw_ctau-1_AllLxyBin.csv


In [14]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 46.15


In [15]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [16]:
# Signal
sig_histo = util.load(f'{prefix}/ctau-1-BDT-based_AllLxy.coffea')[0]

In [17]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,55.0,10.0,1.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-1
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.5,1.0,1.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-1
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.2,0.5,1.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-1
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,52.5,5.0,1.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-1
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,44.0,8.0,1.0,40.0,48.0,0.2,sig_2018_Mchi-44p0_dMchi-8p0_ctau-1
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,33.0,6.0,1.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-1
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,42.0,4.0,1.0,40.0,44.0,0.1,sig_2018_Mchi-42p0_dMchi-4p0_ctau-1
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,31.5,3.0,1.0,30.0,33.0,0.1,sig_2018_Mchi-31p5_dMchi-3p0_ctau-1
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,22.0,4.0,1.0,20.0,24.0,0.2,sig_2018_Mchi-22p0_dMchi-4p0_ctau-1
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,21.0,2.0,1.0,20.0,22.0,0.1,sig_2018_Mchi-21p0_dMchi-2p0_ctau-1


In [18]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'BDT v2 Loose WP']

### Efficiency

In [19]:
pd.options.display.float_format = '{:,.6f}'.format

In [20]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [21]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [22]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.033826,0.006982,0.005961,0.005695,0.005695,0.005695,0.005651,0.003160
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,0.014484,0.001842,0.001519,0.001421,0.001418,0.001418,0.001404,0.000697
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,0.014782,0.002143,0.001894,0.001811,0.001811,0.001811,0.001794,0.000719
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,0.036318,0.006259,0.005421,0.005217,0.005217,0.005217,0.005197,0.002415
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,0.026904,0.005099,0.004306,0.004165,0.004161,0.004161,0.004135,0.002132
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,0.020878,0.003848,0.003369,0.003242,0.003235,0.003235,0.003195,0.001824
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,0.030555,0.005051,0.004338,0.004133,0.004127,0.004127,0.004103,0.001978
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,0.023214,0.003481,0.002980,0.002881,0.002881,0.002881,0.002855,0.001288
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,0.016920,0.002932,0.002531,0.002386,0.002386,0.002386,0.002357,0.001254
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,0.018942,0.002826,0.002381,0.002275,0.002273,0.002273,0.002253,0.001053


In [23]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_ctau-1-BDT-based_Signal_ctau-1_samples_Efficiency_ctau-1_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_ctau-1-BDT-based_Signal_ctau-1_samples_Efficiency_ctau-1_AllLxyBin.csv


### Weighted Event count

In [24]:
pd.options.display.float_format = '{:,.2f}'.format

In [25]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [26]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [27]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,19.17
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,132.25
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","3,032.37"
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,436.31
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,40.64
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,"49,723.24","9,164.66","8,024.74","7,722.24","7,704.44","7,704.44","7,609.52","4,344.52"
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33","1,048.15"
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,"33,794.92","5,067.47","4,338.18","4,193.56","4,193.56","4,193.56","4,156.59","1,875.15"
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,73.76
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"30,374.51","4,531.00","3,817.78","3,648.14","3,644.60","3,644.60","3,612.88","1,688.01"


In [28]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_ctau-1-BDT-based_Signal_ctau-1_samples_EventCount_Weighted_ctau-1_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_ctau-1-BDT-based_Signal_ctau-1_samples_EventCount_Weighted_ctau-1_AllLxyBin.csv


In [29]:
# without m1 = 30 GeV
ct = 1.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,19.17
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,132.25
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","3,032.37"
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,436.31
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,40.64
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33","1,048.15"
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,73.76
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"30,374.51","4,531.00","3,817.78","3,648.14","3,644.60","3,644.60","3,612.88","1,688.01"
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,"1,394.26",207.15,178.78,170.89,170.89,170.89,169.82,83.43
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,"39,395.49","5,811.03","5,006.12","4,654.25","4,648.89","4,648.89","4,616.98","1,951.69"


### Raw count

In [30]:
pd.options.display.float_format = '{:,.0f}'.format

In [31]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [32]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [33]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,761,157,134,128,128,128,127,71
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,4261,542,447,418,417,417,413,205
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3558,516,456,436,436,436,432,173
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,14604,2517,2180,2098,2098,2098,2090,971
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,8179,1550,1309,1266,1265,1265,1257,648
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,8377,1544,1352,1301,1298,1298,1282,732
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,10151,1678,1441,1373,1371,1371,1363,657
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,10056,1508,1291,1248,1248,1248,1237,558
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,7098,1230,1062,1001,1001,1001,989,526
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,8600,1283,1081,1033,1032,1032,1023,478


In [34]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_ctau-1-BDT-based_Signal_ctau-1_samples_EventCount_Raw_ctau-1_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_ctau-1-BDT-based_Signal_ctau-1_samples_EventCount_Raw_ctau-1_AllLxyBin.csv


In [35]:
nBkg

46.14937863220887

In [36]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [37]:
pd.options.display.float_format = '{:,.4f}'.format

In [38]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [39]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,132.2518,5.0000,0.2000,1.0000,46.1494
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"3,032.3719",5.0000,0.1000,1.0000,46.1494
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,83.4290,10.0000,0.2000,1.0000,46.1494
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,"1,951.6912",10.0000,0.1000,1.0000,46.1494
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,73.7553,20.0000,0.2000,1.0000,46.1494
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"1,688.0138",20.0000,0.1000,1.0000,46.1494
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,"4,344.5185",30.0000,0.2000,1.0000,46.1494
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,"1,875.1512",30.0000,0.1000,1.0000,46.1494
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.6448,40.0000,0.2000,1.0000,46.1494
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"1,048.1523",40.0000,0.1000,1.0000,46.1494


In [40]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,132.2518,46.1494,19.4679
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,"3,032.3719",46.1494,446.3747
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0000,0.2000,1.0000,83.4290,46.1494,12.2810
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0000,0.1000,1.0000,"1,951.6912",46.1494,287.2951
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0000,0.2000,1.0000,73.7553,46.1494,10.8570
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0000,0.1000,1.0000,"1,688.0138",46.1494,248.4809
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0000,0.2000,1.0000,"4,344.5185",46.1494,639.5268
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0000,0.1000,1.0000,"1,875.1512",46.1494,276.0281
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0000,0.2000,1.0000,40.6448,46.1494,5.9830
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0000,0.1000,1.0000,"1,048.1523",46.1494,154.2913


In [41]:
outdir = './csv/SR_Yields/'
outname = 'SR_ctau-1-BDT-based_ctau-1_AllLxy_S_over_B'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_ctau-1-BDT-based_AllLxy_S_over_B.csv.csv


### ctau = 1mm signal points

In [42]:
# S/sqrt(B) in ctau = 1mm, delta = 0.1
ct = 1.0
delta = 0.1

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,"3,032.3719",46.1494,446.3747
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0000,0.1000,1.0000,"1,951.6912",46.1494,287.2951
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0000,0.1000,1.0000,"1,688.0138",46.1494,248.4809
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0000,0.1000,1.0000,"1,875.1512",46.1494,276.0281
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0000,0.1000,1.0000,"1,048.1523",46.1494,154.2913
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,50.0000,0.1000,1.0000,436.3098,46.1494,64.2262


In [43]:
# S/sqrt(B) in ctau = 1mm, delta = 0.2
ct = 1.0
delta = 0.2

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,132.2518,46.1494,19.4679
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0000,0.2000,1.0000,83.4290,46.1494,12.2810
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0000,0.2000,1.0000,73.7553,46.1494,10.8570
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0000,0.2000,1.0000,"4,344.5185",46.1494,639.5268
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0000,0.2000,1.0000,40.6448,46.1494,5.9830
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,50.0000,0.2000,1.0000,19.1678,46.1494,2.8216


# SR BDT-ctau-1: High Lxy (Lxy > 0.5)

## Background

In [34]:
prefix = './coffea/skimmed/'
suffix = '_ctau-1-BDT-based_HighLxy'

In [35]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [36]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000003,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000015,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000034,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000066,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000173,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,7.84,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,139.54,0.00
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,48.18,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,103.12,0.29
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,0.93,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39","6,454.21",18.02
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76","3,694.44",3.33
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50","10,448.27",21.64


In [38]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
QCD,12311,381,160,159,159,158,3,1,0
Top,49818,4688,3058,2348,2337,2336,2006,972,0
DY,205,13,11,11,11,11,9,2,0
Diboson,18067,1234,915,860,852,848,767,419,1
Triboson,2257,170,116,106,105,104,92,50,0
W+jets,697657,48425,37015,35472,35415,35377,29090,16594,58
Z+jets,1281459,67427,52627,50986,50934,50903,44480,26108,27
Total,2061774,122338,93902,89942,89813,89737,76447,44146,86


In [39]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 21.64


## Signal

In [40]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [41]:
# Signal
sig_histo = util.load(f'{prefix}/ctau-1-BDT-based_HighLxy.coffea')[0]

In [42]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,55.0,10.0,1.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-1
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.5,1.0,1.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-1
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.2,0.5,1.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-1
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,52.5,5.0,1.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-1
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,44.0,8.0,1.0,40.0,48.0,0.2,sig_2018_Mchi-44p0_dMchi-8p0_ctau-1
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,42.0,4.0,1.0,40.0,44.0,0.1,sig_2018_Mchi-42p0_dMchi-4p0_ctau-1
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,33.0,6.0,1.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-1
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,31.5,3.0,1.0,30.0,33.0,0.1,sig_2018_Mchi-31p5_dMchi-3p0_ctau-1
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,21.0,2.0,1.0,20.0,22.0,0.1,sig_2018_Mchi-21p0_dMchi-2p0_ctau-1
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,22.0,4.0,1.0,20.0,24.0,0.2,sig_2018_Mchi-22p0_dMchi-4p0_ctau-1


In [43]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'Lxy > 0.5',
 'BDT v2 Loose WP']

### Efficiency

In [44]:
pd.options.display.float_format = '{:,.6f}'.format

In [45]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [46]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [47]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.033826,0.006982,0.005961,0.005695,0.005695,0.005695,0.005651,0.001601,0.000356
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,0.014484,0.001842,0.001519,0.001421,0.001418,0.001418,0.001404,0.000915,0.000500
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,0.014782,0.002143,0.001894,0.001811,0.001811,0.001811,0.001794,0.001159,0.000552
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,0.036318,0.006259,0.005421,0.005217,0.005217,0.005217,0.005197,0.001761,0.000405
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,0.026904,0.005099,0.004306,0.004165,0.004161,0.004161,0.004135,0.001303,0.000303
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,0.030555,0.005051,0.004338,0.004133,0.004127,0.004127,0.004103,0.001595,0.000467
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,0.020878,0.003848,0.003369,0.003242,0.003235,0.003235,0.003195,0.001094,0.000386
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,0.023214,0.003481,0.002980,0.002881,0.002881,0.002881,0.002855,0.001237,0.000381
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,0.018942,0.002826,0.002381,0.002275,0.002273,0.002273,0.002253,0.001159,0.000478
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,0.016920,0.002932,0.002531,0.002386,0.002386,0.002386,0.002357,0.001015,0.000396


### Weighted Event count

In [48]:
pd.options.display.float_format = '{:,.2f}'.format

In [49]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [50]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [51]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,9.71,2.16
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,173.54,94.84
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","4,890.41","2,331.15"
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,318.13,73.25
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,24.83,5.77
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33",845.45,247.24
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,"49,723.24","9,164.66","8,024.74","7,722.24","7,704.44","7,704.44","7,609.52","2,606.35",920.02
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,"33,794.92","5,067.47","4,338.18","4,193.56","4,193.56","4,193.56","4,156.59","1,801.23",554.81
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"30,374.51","4,531.00","3,817.78","3,648.14","3,644.60","3,644.60","3,612.88","1,857.71",766.48
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,59.73,23.27


In [52]:
# without m1 = 30 GeV
ct = 1.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,9.71,2.16
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,173.54,94.84
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","4,890.41","2,331.15"
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,318.13,73.25
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,24.83,5.77
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33",845.45,247.24
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"30,374.51","4,531.00","3,817.78","3,648.14","3,644.60","3,644.60","3,612.88","1,857.71",766.48
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,59.73,23.27
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,"1,394.26",207.15,178.78,170.89,170.89,170.89,169.82,92.39,45.67
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,"39,395.49","5,811.03","5,006.12","4,654.25","4,648.89","4,648.89","4,616.98","2,724.22","1,199.76"


### Raw count

In [53]:
pd.options.display.float_format = '{:,.0f}'.format

In [54]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [55]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [56]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,761,157,134,128,128,128,127,36,8
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,4261,542,447,418,417,417,413,269,147
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3558,516,456,436,436,436,432,279,133
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,14604,2517,2180,2098,2098,2098,2090,708,163
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,8179,1550,1309,1266,1265,1265,1257,396,92
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,10151,1678,1441,1373,1371,1371,1363,530,155
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,8377,1544,1352,1301,1298,1298,1282,439,155
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,10056,1508,1291,1248,1248,1248,1237,536,165
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,8600,1283,1081,1033,1032,1032,1023,526,217
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,7098,1230,1062,1001,1001,1001,989,426,166


In [57]:
nBkg

21.638266045940078

In [58]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [59]:
pd.options.display.float_format = '{:,.4f}'.format

df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [60]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,94.8438,5.0000,0.2000,1.0000,21.6383
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"2,331.1456",5.0000,0.1000,1.0000,21.6383
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,45.6732,10.0000,0.2000,1.0000,21.6383
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,"1,199.7602",10.0000,0.1000,1.0000,21.6383
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,766.4788,20.0000,0.1000,1.0000,21.6383
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,23.2723,20.0000,0.2000,1.0000,21.6383
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,920.0155,30.0000,0.2000,1.0000,21.6383
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,554.8139,30.0000,0.1000,1.0000,21.6383
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,5.7681,40.0000,0.2000,1.0000,21.6383
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,247.2436,40.0000,0.1000,1.0000,21.6383


In [61]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,94.8438,21.6383,20.3891
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,"2,331.1456",21.6383,501.1390
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0000,0.2000,1.0000,45.6732,21.6383,9.8186
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0000,0.1000,1.0000,"1,199.7602",21.6383,257.9189
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0000,0.1000,1.0000,766.4788,21.6383,164.7741
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0000,0.2000,1.0000,23.2723,21.6383,5.0030
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0000,0.2000,1.0000,920.0155,21.6383,197.7807
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0000,0.1000,1.0000,554.8139,21.6383,119.2713
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0000,0.2000,1.0000,5.7681,21.6383,1.2400
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0000,0.1000,1.0000,247.2436,21.6383,53.1513


# SR BDT-ctau-1: Low Lxy (Lxy <= 0.5)

## Background

In [63]:
prefix = './coffea/skimmed/'
suffix = '_ctau-1-BDT-based_LowLxy'

In [64]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [65]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000004,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000012,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000042,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000048,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000120,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [66]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,624.41,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,154.46,0.33
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,407.47,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,86.23,0.49
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,1.16,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39","4,750.18",16.93
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76","2,562.32",6.77
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50","8,586.23",24.51


In [67]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
QCD,12311,381,160,159,159,158,3,2,0
Top,49818,4688,3058,2348,2337,2336,2006,1034,3
DY,205,13,11,11,11,11,9,7,0
Diboson,18067,1234,915,860,852,848,767,348,2
Triboson,2257,170,116,106,105,104,92,42,0
W+jets,697657,48425,37015,35472,35415,35377,29090,12496,52
Z+jets,1281459,67427,52627,50986,50934,50903,44480,18372,48
Total,2061774,122338,93902,89942,89813,89737,76447,32301,105


In [68]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 24.51


## Signal

In [69]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [70]:
# Signal
sig_histo = util.load(f'{prefix}/ctau-1-BDT-based_LowLxy.coffea')[0]

In [71]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,55.0,10.0,1.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-1
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.2,0.5,1.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-1
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.5,1.0,1.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-1
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,52.5,5.0,1.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-1
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,44.0,8.0,1.0,40.0,48.0,0.2,sig_2018_Mchi-44p0_dMchi-8p0_ctau-1
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,33.0,6.0,1.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-1
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,42.0,4.0,1.0,40.0,44.0,0.1,sig_2018_Mchi-42p0_dMchi-4p0_ctau-1
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,31.5,3.0,1.0,30.0,33.0,0.1,sig_2018_Mchi-31p5_dMchi-3p0_ctau-1
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,22.0,4.0,1.0,20.0,24.0,0.2,sig_2018_Mchi-22p0_dMchi-4p0_ctau-1
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,11.0,2.0,1.0,10.0,12.0,0.2,sig_2018_Mchi-11p0_dMchi-2p0_ctau-1


In [72]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'Lxy <= 0.5',
 'BDT v2 Loose WP']

### Efficiency

In [73]:
pd.options.display.float_format = '{:,.6f}'.format

In [74]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [75]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [76]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.033826,0.006982,0.005961,0.005695,0.005695,0.005695,0.005651,0.004049,0.002492
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,0.014782,0.002143,0.001894,0.001811,0.001811,0.001811,0.001794,0.000635,0.000141
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,0.014484,0.001842,0.001519,0.001421,0.001418,0.001418,0.001404,0.000489,0.000187
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,0.036318,0.006259,0.005421,0.005217,0.005217,0.005217,0.005197,0.003437,0.001887
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,0.026904,0.005099,0.004306,0.004165,0.004161,0.004161,0.004135,0.002833,0.001731
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,0.020878,0.003848,0.003369,0.003242,0.003235,0.003235,0.003195,0.002101,0.001376
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,0.030555,0.005051,0.004338,0.004133,0.004127,0.004127,0.004103,0.002508,0.001424
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,0.023214,0.003481,0.002980,0.002881,0.002881,0.002881,0.002855,0.001618,0.000859
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,0.016920,0.002932,0.002531,0.002386,0.002386,0.002386,0.002357,0.001342,0.000839
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,0.014729,0.002188,0.001889,0.001805,0.001805,0.001805,0.001794,0.000818,0.000370


### Weighted Event count

In [77]:
pd.options.display.float_format = '{:,.2f}'.format

In [78]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [79]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [80]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,24.56,15.12
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","2,681.89",596.14
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,92.86,35.47
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,620.99,341.04
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,54.00,32.99
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,"49,723.24","9,164.66","8,024.74","7,722.24","7,704.44","7,704.44","7,609.52","5,003.17","3,276.03"
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33","1,328.88",754.66
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,"33,794.92","5,067.47","4,338.18","4,193.56","4,193.56","4,193.56","4,156.59","2,355.36","1,249.79"
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,78.94,49.36
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,"1,394.26",207.15,178.78,170.89,170.89,170.89,169.82,77.43,34.98


In [81]:
# without m1 = 30 GeV
ct = 1.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,24.56,15.12
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","2,681.89",596.14
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,92.86,35.47
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,620.99,341.04
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,54.00,32.99
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33","1,328.88",754.66
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,78.94,49.36
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,"1,394.26",207.15,178.78,170.89,170.89,170.89,169.82,77.43,34.98
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"30,374.51","4,531.00","3,817.78","3,648.14","3,644.60","3,644.60","3,612.88","1,755.18",854.45
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,"39,395.49","5,811.03","5,006.12","4,654.25","4,648.89","4,648.89","4,616.98","1,892.76",682.53


### Raw count

In [82]:
pd.options.display.float_format = '{:,.0f}'.format

In [83]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [84]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [85]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,761,157,134,128,128,128,127,91,56
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3558,516,456,436,436,436,432,153,34
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,4261,542,447,418,417,417,413,144,55
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,14604,2517,2180,2098,2098,2098,2090,1382,759
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,8179,1550,1309,1266,1265,1265,1257,861,526
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,8377,1544,1352,1301,1298,1298,1282,843,552
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,10151,1678,1441,1373,1371,1371,1363,833,473
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,10056,1508,1291,1248,1248,1248,1237,701,372
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,7098,1230,1062,1001,1001,1001,989,563,352
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,6536,971,838,801,801,801,796,363,164


In [86]:
nBkg

24.511112586268794

In [87]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [88]:
pd.options.display.float_format = '{:,.4f}'.format

df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [89]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,596.1353,5.0000,0.1000,1.0000,24.5111
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,35.4669,5.0000,0.2000,1.0000,24.5111
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,34.9821,10.0000,0.2000,1.0000,24.5111
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,682.5256,10.0000,0.1000,1.0000,24.5111
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,49.3629,20.0000,0.2000,1.0000,24.5111
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,854.4492,20.0000,0.1000,1.0000,24.5111
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,"3,276.0317",30.0000,0.2000,1.0000,24.5111
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,"1,249.7859",30.0000,0.1000,1.0000,24.5111
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,32.9949,40.0000,0.2000,1.0000,24.5111
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,754.6598,40.0000,0.1000,1.0000,24.5111


In [90]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,596.1353,24.5111,120.4102
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,35.4669,24.5111,7.1638
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0000,0.2000,1.0000,34.9821,24.5111,7.0658
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0000,0.1000,1.0000,682.5256,24.5111,137.8597
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0000,0.2000,1.0000,49.3629,24.5111,9.9705
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0000,0.1000,1.0000,854.4492,24.5111,172.5857
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0000,0.2000,1.0000,"3,276.0317",24.5111,661.7083
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0000,0.1000,1.0000,"1,249.7859",24.5111,252.4376
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0000,0.2000,1.0000,32.9949,24.5111,6.6645
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0000,0.1000,1.0000,754.6598,24.5111,152.4297
